In [1]:
import math
import tensorflow as tf
import numpy as np

In [2]:
class RBM(object):
    
    def __init__(self, input_size, output_size, lr=1.0, batchsize=100):
        self._input_size = input_size
        self._output_size = output_size
        self.learning_rate = lr
        self.batchsize = batchsize
        
        self.w = tf.zeros([input_size, output_size], tf.float32)
        self.hb = tf.zeros([output_size], tf.float32)
        self.vb = tf.zeros([input_size], tf.float32)

    def prob_h_given_v(self, visible, w, hb):
        return tf.nn.sigmoid(tf.matmul(visible, w) + hb)

    def prob_v_given_h(self, hidden, w, vb):
        return tf.nn.sigmoid(tf.matmul(hidden, tf.transpose(w)) + vb)
    
    def sample_prob(self, probs):
        return tf.nn.relu(tf.sign(probs - tf.random.uniform(tf.shape(probs))))

    def train(self, X, epochs=5):               
        loss = []
        
        for epoch in range(epochs):
            for start, end in zip(range(0, len(X), self.batchsize),range(self.batchsize,len(X), self.batchsize)):
                batch = X[start:end]
                    
                h0 = self.sample_prob(self.prob_h_given_v(batch, self.w, self.hb))
                v1 = self.sample_prob(self.prob_v_given_h(h0, self.w, self.vb))
                h1 = self.prob_h_given_v(v1, self.w, self.hb)
                    
                positive_grad = tf.matmul(tf.transpose(batch), h0)
                negative_grad = tf.matmul(tf.transpose(v1), h1)
                    
                self.w = self.w + self.learning_rate *(positive_grad - negative_grad) / tf.dtypes.cast(tf.shape(batch)[0],tf.float32)
                self.vb = self.vb +  self.learning_rate * tf.reduce_mean(batch - v1, 0)
                self.hb = self.hb +  self.learning_rate * tf.reduce_mean(h0 - h1, 0)
                    
            err = tf.reduce_mean(tf.square(batch - v1))
        
            print ('Epoch: %d' % epoch,'reconstruction error: %f' % err)
            
            loss.append(err)
                    
        return loss
        
    def rbm_output(self, X):
        out = tf.nn.sigmoid(tf.matmul(X, self.w) + self.hb)
        
        return out
    
    def rbm_reconstruct(self,X):
        h = tf.nn.sigmoid(tf.matmul(X, self.w) + self.hb)
        reconstruct = tf.nn.sigmoid(tf.matmul(h, tf.transpose(self.w)) + self.vb)
        
        return reconstruct            

In [3]:
RESHAPED = 784
NB_CLASSES = 10

(train_data, Y_train), (test_data, Y_test) =  tf.keras.datasets.mnist.load_data()
train_data = train_data/np.float32(255)
train_data = np.reshape(train_data, (train_data.shape[0], RESHAPED))
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

test_data = test_data/np.float32(255)
test_data = np.reshape(test_data, (test_data.shape[0], RESHAPED))

In [4]:
RBM_hidden_sizes = [500, 200 , 50 ]

inpX = train_data

rbm_list = []

input_size = train_data.shape[1]

for i, size in enumerate(RBM_hidden_sizes):
    print ('RBM: ',i,' ',input_size,'->', size)

    rbm_list.append(RBM(input_size, size))
    input_size = size

RBM:  0   784 -> 500
RBM:  1   500 -> 200
RBM:  2   200 -> 50


In [5]:
for rbm in rbm_list:
    print ('New RBM:')

    rbm.train(tf.cast(inpX,tf.float32)) 
    
    inpX = rbm.rbm_output(inpX)

New RBM:
Epoch: 0 reconstruction error: 0.058193
Epoch: 1 reconstruction error: 0.051587
Epoch: 2 reconstruction error: 0.047794
Epoch: 3 reconstruction error: 0.045730
Epoch: 4 reconstruction error: 0.047502
New RBM:
Epoch: 0 reconstruction error: 0.028351
Epoch: 1 reconstruction error: 0.024901
Epoch: 2 reconstruction error: 0.022829
Epoch: 3 reconstruction error: 0.022887
Epoch: 4 reconstruction error: 0.022835
New RBM:
Epoch: 0 reconstruction error: 0.056835
Epoch: 1 reconstruction error: 0.049966
Epoch: 2 reconstruction error: 0.050687
Epoch: 3 reconstruction error: 0.049387
Epoch: 4 reconstruction error: 0.049573
